In [48]:
import quandl
import pandas as pd
import numpy as np
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame, TimeFrameUnit
import datetime as dt
import os
import hvplot.pandas


In [49]:
# Alpaca API keys, set in env 
api_key = os.environ.get('APCA_API_KEY_ID')
api_secret = os.environ.get('APCA_API_SECRET_KEY')

# Quandl API
quandl.ApiConfig.api_key = os.environ.get('QUANDL_API')

# Alpaca API endpoint
base_url = 'https://data.alpaca.markets/v2'

# instantiate REST alpaca API
api = REST(api_key, api_secret, base_url, api_version='v2')




In [50]:
# Reading trading dates CSV
date_csv = pd.read_csv("trade_dates.csv")

In [53]:
# Back testing for specefic date : weekends and or holidays will return error "type object 'object' has no attribute 'dtype'". 
# Usage per trade will be calculated by account size over total trades per day 
          
def backtest_date(account_size, stop_loss_percent, trades_per_day):
  
    trade_USD = account_size/trades_per_day
    profit_for_day = []
    stoped = []
    tickers = []
    day_counter = 0

    for d in range(len(date_csv)-1):        
        # Generate quandl data
        accending =  quandl.get_table('NDAQ/RTAT', date=date_csv["date"][d]).sort_values(by = "sentiment", axis = 0, ascending=False, ignore_index = True).head(trades_per_day)
        day_counter += 1

        # Calculate profit 
        for i in range(len(accending)):

            # Get tiker data and calculate profit for given day
            min_15_close = api.get_bars(accending["ticker"][i], TimeFrame(15, TimeFrameUnit.Minute), date_csv["date"][d+1], adjustment='raw', limit = 1).df
            day_close = api.get_bars(accending["ticker"][i], TimeFrame.Day, date_csv["date"][d+1], adjustment='raw', limit = 1).df
            profit = round((day_close["close"][0] - min_15_close["close"][0])*((1/min_15_close["close"][0])*trade_USD), 2)
            daily_percent_drop = ((day_close["close"][0] - min_15_close["close"][0])/min_15_close["close"][0])*100
            tickers.append(accending["ticker"][i])

            # Calculate is stop loss was hit or trade PNL         
            if profit != 0:
                if daily_percent_drop < (-1*stop_loss_percent):
                    profit_for_day.append(trade_USD*((-stop_loss_percent/100)))
                    stoped.append(accending["ticker"][i])
                else:
                    profit_for_day.append(profit)

    print(f"day counter = {day_counter}.")

    return profit_for_day

In [ ]:
# Running function to provide data for plotting and profit
backtest_date(100000, 1, 10)
pnl = backtest_date(100000, 1, 10)

sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/KDFI/bars 3 more time(s)...


In [ ]:
# Calculating start and end date of trading days
start = date_csv["date"][0]
end = date_csv["date"][len(date_csv["date"])-1]

# Calculating cumsum profit
profit_df = pd.DataFrame(pnl).cumsum()

# Plotting profit data and printing key data
plot_pnl = profit_df.hvplot(ylabel = "PNL", xlabel = "Trade Index", title = f"Buzzscore Backtest 5 Trades Daily  -  PNL ${round(sum(pnl), 2)}     [{start}"f"  -  {end}]")
print(f"PNL = ${sum(pnl)}")
print(f"Trade Count = {len(pnl)}")
print(f"Max win trade = ${max(pnl)}")
print(f"Max loss trade = ${min(pnl)}")
print(f"AVG trade value = ${round(sum(pnl)/len(pnl),2)}")
print(f"Start/End date = {start}"f" - {end}")
plot_pnl

In [ ]:
std_plot = profit_df.plot(kind = "box")